In [1]:
import os
import os.path as op
import json
import cv2
import base64
import numpy as np
from tqdm import tqdm

from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
from maskrcnn_benchmark.structures.tsv_file_ops import generate_linelist_file
from maskrcnn_benchmark.structures.tsv_file_ops import generate_hw_file
from maskrcnn_benchmark.structures.tsv_file import TSVFile
from maskrcnn_benchmark.data.datasets.utils.image_ops import img_from_base64

In [4]:
orig_root = '/media/stopmosk/data/huawei/datasets_orig/textcaps_orig'

orig_img_train_val_dir = op.join(orig_root, 'train_images')  # Contains train & val images
orig_img_test_dir = op.join(orig_root, 'test_images')

orig_cap_filenames = {split: f'TextCaps_0.1_{split}.json' for split in ['train', 'val', 'test']}

exp_root = '/media/stopmosk/data/huawei/datasets_proc/textcaps_cleaned_nn_json'

# cap_exp = '_caption.json'

In [5]:
caps = {'train': [], 'val': [], 'test': []}

for split in caps.keys():
    cap_filename = op.join(orig_root, orig_cap_filenames[split])
    with open(cap_filename) as fp:
        captions_json = json.load(fp)
    caps[split] = captions_json['data']

# caps['train'][0]

In [6]:
img_list = {split_name: set([item['image_id'] + '.jpg' for item in caps[split_name]]) for split_name in caps.keys()}
for split in img_list:
    print(len(img_list[split]))

21953
3166
3289


In [10]:
orig_root_2 = '../../../../datasets_orig/textcaps_cleaned'

img_list = sorted(os.listdir(op.join(orig_root_2, 'images')))
img_list = [name.split('.')[0] for name in img_list]
#img_key2path = {img_p.split('.')[0]: op.join(orig_root, 'images', img_p) for img_p in img_list}

#sample_id = list(img_key2path.keys())[0]
#print(sample_id, '\n', img_key2path[sample_id])
img_list[0]

'0005dcda3040dab3'

In [7]:
# for split in caps.keys():  
#     rows = []
#     rows_label = []
#     rows_hw = []

#     i = 2
#     for img_p in tqdm(img_list[split]):
#         img_key = img_p.split('.')[0]
#         img_path = op.join(orig_root, f"{'test' if split=='test' else 'train'}_images", img_p)
#         img = cv2.imread(img_path)
#         img_encoded_str = base64.b64encode(cv2.imencode('.jpg', img)[1])
#         row = [img_key, img_encoded_str]
#         # print(row[1][800:900] , flush=True)
#         rows.append(row)

#         height = img.shape[0]
#         width = img.shape[1]
#         row_hw = [img_key, json.dumps([{'height': height, 'width': width}])]
#         rows_hw.append(row_hw)
#         i -= 1
#         if i == 0:
#             break
        
#     exp_encoded_img_file = op.join(exp_root, f'{split}.img.tsv')
#     exp_hw_file = op.join(exp_root, f'{split}.hw.tsv')
#     print(exp_encoded_img_file, flush=True)
#     tsv_writer(rows, exp_encoded_img_file)
#     # tsv_writer(rows_label, label_file)
#     tsv_writer(rows_hw, exp_hw_file)

In [24]:
# caps['val'][0]

In [11]:
def generate_cap_json(split: str):
    captions = []
    cap_idx = 0
    for sample in caps[split]:
        image_id = sample['image_id']
        caption_str = sample['caption_str']

        if image_id not in img_list:
            continue

        captions.append(
            {
                'image_id': image_id,
                'id': cap_idx,
                'caption': caption_str,
            }
        )
        cap_idx +=1
    captions = sorted(captions, key=lambda k: k['image_id'])
    print(captions[:10])
    return captions

In [14]:
# cap_exp = '_caption.json'

all_json = []
for split in ['train', 'val']:
    all_json.extend(generate_cap_json(split))
    # cap_filename = op.join(exp_root, f'{split}_caption.json')
    # with open(cap_filename, 'w') as fp:
    #     json.dump(generate_cap_json(split), fp)
        
cap_filename = op.join(exp_root, 'train_caption.json')
with open(cap_filename, 'w') as fp:
    json.dump(all_json, fp)

[{'image_id': '0005dcda3040dab3', 'id': 1695, 'caption': 'Three women sit at a table for a picture and it has LM on the lower right corner.'}, {'image_id': '0005dcda3040dab3', 'id': 11630, 'caption': 'Three women posing for a photo with the letters LM on the bottom.'}, {'image_id': '0005dcda3040dab3', 'id': 11733, 'caption': 'Three girls pose for a picture on Losmocanos.com'}, {'image_id': '0005dcda3040dab3', 'id': 11778, 'caption': "three girls taking a picture that says 'lm' on the bottom right"}, {'image_id': '0005dcda3040dab3', 'id': 12207, 'caption': 'the photo of the three young women is taken by www.losmocanos.com'}, {'image_id': '0007a5a18213563f', 'id': 14891, 'caption': 'a man named Ludwig has a statue standing of him'}, {'image_id': '0007a5a18213563f', 'id': 14939, 'caption': 'A stone statue of a man says Ludwig Koenig Von Bayern.'}, {'image_id': '0007a5a18213563f', 'id': 15349, 'caption': 'A statue of "Ludwig" is shown in a city square in front of a yellow building.'}, {'im